# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
!ls

event_data  event_datafile_new.csv  images  Project_1B_ Project_Template.ipynb


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)


cluster = Cluster(['localhost'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS projects2 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('projects2')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
QUERY_TEMPLATE = "CREATE TABLE IF NOT EXISTS {} ({}, PRIMARY KEY({}))"
SELECT = "SELECT {} FROM {} WHERE {}"

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

The answer:

```bash
+-----------+---------------------------------+
|   artist  |               song              |
+-----------+---------------------------------+
| Faithless | Music Matters (Mark Knight Dub) |
+-----------+---------------------------------+
```

#### Varibale for question 1

In [11]:
table_name = "songs_info_by_session"
# List of columns in table song as a tuple
cols_1 = ('sessionId','itemInSession','artist','length','song')
col_type = ('int', 'int', 'text', 'float', 'text')
columns = ",".join(["{} {}".format(x,y) for x,y in zip(cols_1, col_type)])
primary_key = "sessionId, itemInSession"

In [12]:
query = QUERY_TEMPLATE.format(table_name, columns, primary_key)
print(query)
try:
    session.execute(query)
except Exception as e:
    print(e)

CREATE TABLE IF NOT EXISTS songs_info_by_session (sessionId int,itemInSession int,artist text,length float,song text, PRIMARY KEY(sessionId, itemInSession))


In [13]:
def insert_query(*list_cols, table='songs'):
    """Generate insert query with specify table and columns.
    
    Parameter
    ---------
    list_cols: args
        List of columns will insert values
    table: str
        Name of table will be inserted
    
    Return:
        SQL query as a string
    """
    c = "{}".format(",".join(x for x in list_cols))
    return f"INSERT INTO {table} ({c})" + " VALUES ({})".format(",".join(["%s"]*len(list_cols)))

In [14]:
# Get header of csv file
with open('event_datafile_new.csv', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    header = next(csvreader)
    f.close()
header_idx = {v:k for k,v in enumerate(header)}

In [15]:
# Check query
insert_query(*cols_1, table=table_name)

'INSERT INTO songs_info_by_session (sessionId,itemInSession,artist,length,song) VALUES (%s,%s,%s,%s,%s)'

In [16]:
for i in cols_1:
    print(f"{i}: {header_idx[i]}")

sessionId: 8
itemInSession: 3
artist: 0
length: 5
song: 9


In [17]:
file = 'event_datafile_new.csv'
query = insert_query(*cols_1, table=table_name)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (int(line[8]),int(line[3]),line[0],float(line[5]),line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [18]:
def gen_result(res):
    """Generate result from cassandra.
    
    Parameter
    ---------
    res: cassandra.cluster.ResultSet
        Result of cassandra query
        
    Return
    SQL result as a table
    """
    table = PrettyTable()
    table.field_names = res.column_names
    for row in res:
        table.add_row(row)
    return table

In [19]:
query = SELECT.format("artist, song", table_name, "sessionId = 338 AND iteminsession = 4")
try:
    results = session.execute(query)
except Exception as error:
    print(error)

In [20]:
print("the artist, song title and song's length in the music app history that was heard during \
sessionId = 338, and itemInSession = 4")
print(gen_result(results))

the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
+-----------+---------------------------------+
|   artist  |               song              |
+-----------+---------------------------------+
| Faithless | Music Matters (Mark Knight Dub) |
+-----------+---------------------------------+


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

The answer

```bash
+-------------------+------------------------------------------------------+-----------+----------+
|       artist      |                         song                         | firstname | lastname |
+-------------------+------------------------------------------------------+-----------+----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie  |   Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie  |   Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie  |   Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie  |   Cruz   |
+-------------------+------------------------------------------------------+-----------+----------+
```

In [21]:
table2 = "songs_info_by_user"
# List of columns in table song as a tuple
cols_2 = ('userId','sessionId','itemInSession','artist','firstName','lastName', 'song')
type_2 = ('int', 'int', 'int', 'text', 'text', 'text', 'text')
columns2 = ",".join(["{} {}".format(x,y) for x,y in zip(cols_2, type_2)])
primary_key2 = "(userId, sessionId), itemInSession"

In [22]:
query = QUERY_TEMPLATE.format(table2, columns2, primary_key2)
query = query + """ WITH CLUSTERING ORDER BY (itemInSession ASC);"""
print(query)
try:
    session.execute(query)
except Exception as e:
    print(e)

CREATE TABLE IF NOT EXISTS songs_info_by_user (userId int,sessionId int,itemInSession int,artist text,firstName text,lastName text,song text, PRIMARY KEY((userId, sessionId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC);


In [23]:
insert_query(*cols_2, table=table2)

'INSERT INTO songs_info_by_user (userId,sessionId,itemInSession,artist,firstName,lastName,song) VALUES (%s,%s,%s,%s,%s,%s,%s)'

In [24]:
for i in cols_2:
    print(f"{i}: {header_idx[i]}")

userId: 10
sessionId: 8
itemInSession: 3
artist: 0
firstName: 1
lastName: 4
song: 9


In [25]:
query = insert_query(*cols_2, table=table2)
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[1],line[4],line[9]))

In [26]:
query = f"""
    SELECT artist, song, firstName, lastName 
    FROM {table2} 
    WHERE userId = 10 AND sessionId = 182
"""
try:
    results = session.execute(query)
except Exception as error:
    print(error)

In [27]:
print(gen_result(results))

+-------------------+------------------------------------------------------+-----------+----------+
|       artist      |                         song                         | firstname | lastname |
+-------------------+------------------------------------------------------+-----------+----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie  |   Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie  |   Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie  |   Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie  |   Cruz   |
+-------------------+------------------------------------------------------+-----------+----------+


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

The answer:

```bash
+------------+----------+
| firstname  | lastname |
+------------+----------+
|    Sara    | Johnson  |
|   Tegan    |  Levine  |
| Jacqueline |  Lynch   |
+------------+----------+
```

In [28]:
table3 = "user_info_by_song"
# List of columns in table song as a tuple
cols_3 = ('userId','song','firstName','lastName')
type_3 = ('int', 'text', 'text', 'text')
columns3 = ",".join(["{} {}".format(x,y) for x,y in zip(cols_3, type_3)])
primary_key3 = "userId, song"

In [29]:
query = QUERY_TEMPLATE.format(table3, columns3, primary_key3)
print(query)

try:
    session.execute(query)
except Exception as e:
    print(e)

                    

CREATE TABLE IF NOT EXISTS user_info_by_song (userId int,song text,firstName text,lastName text, PRIMARY KEY(userId, song))


In [30]:
insert_query(*cols_3, table=table3)

'INSERT INTO user_info_by_song (userId,song,firstName,lastName) VALUES (%s,%s,%s,%s)'

In [31]:
for i in cols_3:
    print(f"{i}: {header_idx[i]}")

userId: 10
song: 9
firstName: 1
lastName: 4


In [32]:
# file = 'event_datafile_new.csv'
query = insert_query(*cols_3, table=table3)
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (int(line[10]),line[9],line[1],line[4]))

In [33]:
query = f"""
    SELECT firstName, lastName 
    FROM {table3} 
    WHERE song = 'All Hands Against His Own'
    ALLOW FILTERING
"""
try:
    results = session.execute(query)
except Exception as error:
    print(error)

In [34]:
print(gen_result(results))

+------------+----------+
| firstname  | lastname |
+------------+----------+
|    Sara    | Johnson  |
|   Tegan    |  Levine  |
| Jacqueline |  Lynch   |
+------------+----------+


### Drop the tables before closing out the sessions

In [35]:
try:
    session.execute("DROP TABLE IF EXISTS songs")
    session.execute("DROP TABLE IF EXISTS users")
    session.execute("DROP TABLE IF EXISTS history")
except Exception as error:
    print(error)

### Close the session and cluster connection¶

In [36]:
session.shutdown()
cluster.shutdown()